In [ ]:
import os
from datasets import Dataset
import dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import evaluate
from ragas.metrics import (
    faithfulness,      # 忠実性
    answer_relevancy,  # 関連性
    context_precision, # 適合率
    context_recall,    # 再現率
)

# .envファイルの読み込み
dotenv.load_dotenv()

# APIキーの設定
key = os.getenv("OPENAI_API_KEY_LLUMINAI")
os.environ["OPENAI_API_KEY"] = key

# --- 1. モデルの明示的な準備 ---
llm = ChatOpenAI(model="gpt-4o-mini") # コスト節約のためminiにしていますが、gpt-4oでもOK
embeddings = OpenAIEmbeddings()

# --- 2. データセットの準備 ---
data = {
    "question": [
        "フランスの首都はどこですか？",
        "Pythonの特徴について教えて。",
        "大谷翔平の2024年の所属チームは？"
    ],
    "answer": [
        "フランスの首都はパリです。", 
        "Pythonは動的型付け言語であり、コンパイルが不要なインタープリタ言語です。",
        "2023年までエンゼルスに所属していましたが、2024年からはドジャースに移籍しました。"
    ],
    "contexts": [
        # 1問目
        ["パリはフランスの首都であり、最大の都市です。", "リヨンは美食の街です。"],
        # 2問目
        ["Pythonはオランダのグイド・ヴァン・ロッサムによって作られました。", "C++はコンパイル言語である。"],
        # 3問目
        ["大谷翔平はドジャースと10年契約を結んだ。", "山本由伸もドジャースに入団した。"]
    ],
    "ground_truth": [
        "パリ",
        "インタープリタ型で動的型付けを持つ、読みやすいプログラミング言語。",
        "ロサンゼルス・ドジャース"
    ]
}

dataset = Dataset.from_dict(data)

# --- 3. 評価の実行 ---
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]

print("評価を開始します")

# llm と embeddings を引数として渡します
results = evaluate(
    dataset=dataset,
    metrics=metrics,
    llm=llm,
    embeddings=embeddings
)

# 総合スコアの表示
print(f"\n総合スコア: {results}")

# --- 4. 結果の表示 ---
df_results = results.to_pandas()

# questionカラムがない場合の安全策
if 'question' not in df_results.columns:
    df_results['question'] = dataset['question']

# 詳細表示
print("\n詳細スコア:")
print(df_results[['question', 'faithfulness', 'context_recall', 'answer_relevancy']])


C:\Users\LoNebula\AppData\Local\Temp\ipykernel_15724\1138446153.py:8: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
C:\Users\LoNebula\AppData\Local\Temp\ipykernel_15724\1138446153.py:8: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import (
C:\Users\LoNebula\AppData\Local\Temp\ipykernel_15724\1138446153.py:8: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import (
C:\Users\LoNebula\AppData\Local\Temp\

評価を開始します...（Embeddingエラーが出ないか確認中）


Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating: 100%|██████████| 12/12 [00:27<00:00,  2.25s/it]



総合スコア: {'faithfulness': 0.3333, 'answer_relevancy': 0.8536, 'context_precision': 0.6667, 'context_recall': 0.6667}

詳細スコア:
             question  faithfulness  context_recall  answer_relevancy
0      フランスの首都はどこですか？           1.0             1.0          0.895648
1   Pythonの特徴について教えて。           0.0             0.0          0.831290
2  大谷翔平の2024年の所属チームは？           0.0             1.0          0.833728
